In [1]:
import urllib.request
import json
import openai
import json
from openai import OpenAI
from openai.types import Completion, CompletionChoice, CompletionUsage
import base64
from pyproj import Proj, Transformer
from random import randrange

In [2]:
import spotipy
import webbrowser
import geocoder
import spotipy.util as util
import random
import requests
import datetime
from io import BytesIO
from PIL import Image

In [3]:
#Get ISS position API
request = urllib.request.Request("http://api.open-notify.org/iss-now.json")
response = urllib.request.urlopen(request)

In [4]:
iss_obj = json.loads(response.read())

In [5]:
# The International Space Station moves at 28,000 km/h 
line = 'ISS'
timestamp = iss_obj['timestamp']
if line == 'ISS':
    latitude=iss_obj['iss_position']['latitude']
    longitude=iss_obj['iss_position']['longitude']
if line == 'python':
    latitude=str(randrange(-90,90))
    longitude=str(randrange(-180,180))
time=datetime.datetime.fromtimestamp(iss_obj['timestamp'])
print(latitude,longitude,time)

21.6856 -22.4450 2023-12-04 11:31:22


In [6]:
with open("keys/city_key.txt", "r") as file:
    city_key = file.read()

In [7]:
#Get location information
requestCity = urllib.request.Request("https://api.opencagedata.com/geocode/v1/json?q="+latitude+","+longitude+"&key="+city_key)
responseCity = urllib.request.urlopen(requestCity)


In [8]:
city_obj = json.loads(responseCity.read())

In [9]:
print(city_obj)

{'documentation': 'https://opencagedata.com/api', 'licenses': [{'name': 'see attribution guide', 'url': 'https://opencagedata.com/credits'}], 'rate': {'limit': 2500, 'remaining': 2468, 'reset': 1701734400}, 'results': [{'annotations': {'DMS': {'lat': "21° 41' 8.16000'' N", 'lng': "22° 26' 42.00000'' W"}, 'MGRS': '27QUD5050798724', 'Maidenhead': 'HL81sq64on', 'Mercator': {'x': -2498565.971, 'y': 2458037.19}, 'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/21.68560/-22.44500&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=21.68560&mlon=-22.44500#map=17/21.68560/-22.44500'}, 'geohash': 'eeb5378pc8ffz0eu153v', 'qibla': 77.72, 'roadinfo': {'drive_on': 'right', 'speed_in': 'km/h'}, 'sun': {'rise': {'apparent': 1701676440, 'astronomical': 1701671760, 'civil': 1701675000, 'nautical': 1701673380}, 'set': {'apparent': 1701715560, 'astronomical': 1701720240, 'civil': 1701717000, 'nautical': 1701718620}}, 'timezone': {'name': 'GMT+1', 'now_in_dst': 0, 'offset_sec': -3600,

In [10]:
#Get the location name
category=city_obj["results"][0]["components"]["_category"]

type=city_obj["results"][0]["components"]["_type"]
if type=='building' or type=='road':
    location=city_obj["results"][0]["components"]["county"]
else:
    location=city_obj["results"][0]["components"][type]
    

print(location)

North Atlantic Ocean


In [11]:
#To check the day time or the night time by the time of sunrise and sunset
loc_sun=city_obj["results"][0]['annotations']['sun']
sunrise=loc_sun['rise']['apparent']
sunset=loc_sun['set']['apparent']
localtime=timestamp+city_obj["results"][0]['annotations']['timezone']['offset_sec']

if localtime>sunrise and localtime<sunset: 
    city_status='day'
else:
    city_status='night'
    
print(city_status)

day


In [12]:

with open("keys/weather_key.txt", "r") as file:
    weather_key = file.read()
    
with open("keys/openai_key.txt", "r") as file:
    openai_key = file.read()

In [13]:
#Get local weather information
weather_url = f'https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={weather_key}'
request_weather = urllib.request.Request(weather_url)
response_weather = urllib.request.urlopen(request_weather)

In [14]:
weather_obj = json.loads(response_weather.read())
weather = weather_obj['weather'][0]['main']
print (weather)

Clouds


In [15]:
#Get key words by OpenAI API
number='15'

client = OpenAI(
   
    api_key= openai_key,
)
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"Generate JSON list of {number} keywords of unique things in {location} in {city_status} time in {weather} weather, with no format, no explanation, no enter.", 

        }
    ],
    model="gpt-3.5-turbo",
)
print(response)


ChatCompletion(id='chatcmpl-8S1SWB4lnqJA90AAPNcL4hGbBB7HL', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='["whales", "seagulls", "fishing boats", "ocean currents", "puffins", "ocean spray", "rainbows", "bioluminescence", "shipwrecks", "whale watching", "sailboats", "dolphins", "storm petrels", "surfing", "kelp forests"]', role='assistant', function_call=None, tool_calls=None))], created=1701689484, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=74, prompt_tokens=39, total_tokens=113))


In [16]:
#Format the response
raw_string = str(response)

if raw_string.find("keywords")>0:
    content_start = raw_string.find('"keywords":' ) + len('"keywords":')
    content_end = raw_string.find("', role=", content_start)-1
else:
    content_start = raw_string.find("content='") + len("content='")
    content_end = raw_string.find("', role=", content_start)
json_content = raw_string[content_start:content_end]


print(json_content)
keywords_list = json.loads(json_content)

print(keywords_list)


["whales", "seagulls", "fishing boats", "ocean currents", "puffins", "ocean spray", "rainbows", "bioluminescence", "shipwrecks", "whale watching", "sailboats", "dolphins", "storm petrels", "surfing", "kelp forests"]
['whales', 'seagulls', 'fishing boats', 'ocean currents', 'puffins', 'ocean spray', 'rainbows', 'bioluminescence', 'shipwrecks', 'whale watching', 'sailboats', 'dolphins', 'storm petrels', 'surfing', 'kelp forests']


In [17]:
with open("keys/spotify_keys.json", "r") as file:
    api_tokens = json.load(file)

In [18]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]
userid = api_tokens["userid"]

In [19]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read ugc-image-upload'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [20]:
sp = spotipy.Spotify(auth=token)

In [21]:
tracks = []

for word in keywords_list:
    searchResults = sp.search(q=word,type='track',limit=1)
    if len(searchResults['tracks']['items'])>0:
        tracks.append(searchResults['tracks']['items'][0]['uri'])

In [22]:


my_playlist = sp.user_playlist_create(user=userid, name=f"Go to {location} with {line}", public=True,
                                      description=f"We are now approching {location}.  Time: {time}  Weather: {weather}")



results = sp.user_playlist_add_tracks(username, my_playlist['id'], tracks)
print(results)
print(my_playlist)



{'snapshot_id': 'MyxkNDA3YmQ3ZTRlMzMyOWU3MTg0OGZiNWZkODI3ODQ2ODUzY2EwNWFk'}
{'collaborative': False, 'description': 'We are now approching North Atlantic Ocean.  Time: 2023-12-04 11:31:22  Weather: Clouds', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4TDqrfWkTvRjMLhHRMZxSS'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/4TDqrfWkTvRjMLhHRMZxSS', 'id': '4TDqrfWkTvRjMLhHRMZxSS', 'images': [], 'name': 'Go to North Atlantic Ocean with ISS', 'owner': {'display_name': 'WangxinLiu', 'external_urls': {'spotify': 'https://open.spotify.com/user/31mvdqp55nskpk66v4nzpyea42ci'}, 'href': 'https://api.spotify.com/v1/users/31mvdqp55nskpk66v4nzpyea42ci', 'id': '31mvdqp55nskpk66v4nzpyea42ci', 'type': 'user', 'uri': 'spotify:user:31mvdqp55nskpk66v4nzpyea42ci'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSw3N2UyN2MyMmM0YTViY2ViMDFiMzY4NzRhNDVkMmIwZjA1Yjc1YzQx', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/4TDqrfWkTvRjMLh

In [23]:
def latlon_to_bbox(lat, lon, buffer_meters=randrange(5000,9000)):
    # Define the WGS84 coordinate system (latitude and longitude)
    wgs84 = Proj('epsg:4326')

    # Convert latitude and longitude to UTM coordinates
    utm_zone, _ = get_utm_zone(float(lat), float(lon))  # Convert to float here
    utm_proj = Proj(proj='utm', zone=utm_zone, ellps='WGS84')
    
    transformer = Transformer.from_proj(wgs84, utm_proj)
    x, y = transformer.transform(float(lat), float(lon))  # Convert to float here

    # Create BBOX
    bbox = (
        x - buffer_meters,
        y - buffer_meters,
        x + buffer_meters,
        y + buffer_meters
    )
    bbox_str = ','.join(map(str, bbox))
    return bbox_str

def get_utm_zone(lat, lon):
    # Determine UTM zone based on longitude
    zone = int((float(lon) + 180) / 6) + 1
    hemisphere = 'north' if float(lat) >= 0 else 'south'
    return zone, hemisphere



bbox = latlon_to_bbox(latitude, longitude)
print(bbox)

341691.748084354,2389908.6198864314,359323.748084354,2407540.6198864314


In [24]:
with open("keys/sentinel_keys.json", "r") as file:
    api_tokens = json.load(file)
sentinel_API = api_tokens["client_API"]

In [25]:
url=f"https://services.sentinel-hub.com/ogc/wms/{sentinel_API}?REQUEST=GetMap&BBOX={bbox}&LAYERS=NATURAL-COLOR&MAXCC=60&WIDTH=640&HEIGHT=640&FORMAT=image/jpeg&TIME=2023-12-04"
webbrowser.open(url)
response = requests.get(url)

In [26]:
if response.status_code == 200:
    # Create an Image object from the content
    img = Image.open(BytesIO(response.content))

    # Save the image to a file
    img.save(f'image.jpg')

    print("Image downloaded successfully.")
else:
    print(f"Failed to download image. Status code: {response.status_code}")

Image downloaded successfully.


In [27]:
playlist_id =  my_playlist['id']

image_path = f'image.jpg'

# load image
with open(image_path, 'rb') as image_file:
    image_data = image_file.read()
    base64_image_data = base64.b64encode(image_data).decode('utf-8')

In [28]:
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'image/jpg'
}

url = f'https://api.spotify.com/v1/playlists/{playlist_id}/images'

# change cover
response = requests.put(url, headers=headers, data=base64_image_data)

In [29]:
webbrowser.open(my_playlist['external_urls']['spotify'])

True

In [30]:
##